In [1]:

# create a spam email filter using Naive Bayes
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB



In [2]:
#import the dataset
spam_df = pd.read_csv('./SpamEmail.csv')

In [3]:
#inspect the dataset
spam_df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4514                         Xin lỗi tôi sẽ gọi lại sau   
spam         747    639  Vui lòng gọi cho đại diện dịch vụ khách hàng c...   

               
         freq  
Category       
ham        30  
spam        4

In [4]:
#turn spam/ham into numerical values , create a new column call 'spam'
spam_df['spam'] = spam_df['Category'].apply(lambda x: 1 if x == 'spam' else 0)
spam_df

,Category,Message,spam
0,ham,"Hãy đến cho đến Jurong Point, điên rồ .. chỉ c...",0
1,ham,Ok lar ... nói đùa wif u oni ...,0
2,spam,Nhập miễn phí trong 2 WKLY Comp để giành chiến...,1
3,ham,Bạn không nói quá sớm ... bạn đã nói ...,0
4,ham,"Không, tôi không nghĩ anh ấy đến USF, anh ấy s...",0
...,...,...,...
5567,spam,Đây là lần thứ 2 chúng tôi đã thử 2 liên hệ u....,1
5568,ham,Ã¼ b sẽ đến nhà Esplanade?,0
5569,ham,"Đáng tiếc, * đã có tâm trạng cho điều đó. Vì v...",0
5570,ham,Anh chàng đã làm một số chó cái nhưng tôi đã h...,0


In [5]:
#Pre-processing
# Convert all to regular letters and Remove extra spaces.
spam_df['Message'] = spam_df['Message'].str.lower().str.replace('\s+', ' ').str.replace('\d+', '')
spam_df





C:\Users\IBM\AppData\Local\Temp\ipykernel_2452\160975084.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  spam_df['Message'] = spam_df['Message'].str.lower().str.replace('\s+', ' ').str.replace('\d+', '')


,Category,Message,spam
0,ham,"hãy đến cho đến jurong point, điên rồ .. chỉ c...",0
1,ham,ok lar ... nói đùa wif u oni ...,0
2,spam,nhập miễn phí trong wkly comp để giành chiến ...,1
3,ham,bạn không nói quá sớm ... bạn đã nói ...,0
4,ham,"không, tôi không nghĩ anh ấy đến usf, anh ấy s...",0
...,...,...,...
5567,spam,đây là lần thứ chúng tôi đã thử liên hệ u. b...,1
5568,ham,ã¼ b sẽ đến nhà esplanade?,0
5569,ham,"đáng tiếc, * đã có tâm trạng cho điều đó. vì v...",0
5570,ham,anh chàng đã làm một số chó cái nhưng tôi đã h...,0


In [6]:
#create train and test split
X_train, X_test, y_train, y_test = train_test_split(spam_df['Message'], spam_df['spam'], test_size=0.25)
X_train.describe()

count                           4179
unique                          3895
top       xin lỗi tôi sẽ gọi lại sau
freq                              25
Name: Message, dtype: object

In [7]:
# The first stage is used for pre-processing and analyzing the received  Vietnamese email into the single words and the compound words. 
# The second stage applies the NaIve Bayesian algorithm basing on the analyzed tokens to calculate the probability of spam or ham email. 
# The last stage is the token self-learning process after the kinds of email such as spam, or ham are identified.

X_train, X_test, y_train, y_test = train_test_split(spam_df['Message'], spam_df['spam'], test_size=0.25)


In [8]:
#find word count and store data as a matrix
cv = CountVectorizer() 
X_train_count = cv.fit_transform(X_train.values) 
X_train_count.toarray()


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [9]:
#train  model
model = MultinomialNB() #multinomial naive bayes is used for classification with discrete features (e.g., word counts for text classification)
model.fit(X_train_count, y_train) #fitting the model

MultinomialNB()

In [10]:
ham_email =[ 'Xin lỗi tôi sẽ gọi lại sau',
            'Ngay cả anh trai tôi cũng không thích nói chuyện với tôi',
            'bạn rảnh không?']
ham_email_count = cv.transform(ham_email)
model.predict(ham_email_count)


array([0, 0, 0], dtype=int64)

In [11]:
spam_email = ['SĂN DEAL HOT LG MỪNG BLACK FRIDAY',
              'Nhập miễn phí 2 khoa hoc cua hoc vien',
              'KHẨN CẤP! Bạn đã giành được tư cách thành viên miễn phí trong 1 tuần trong giải thưởng giải thưởng trị giá 100.000 bảng của chúng tôi! TXT Từ: Yêu cầu NO: 81010 T & C www.dbuk.net LCCLTD POBOX 4403LDNW1A7RW18']

spam_email_count = cv.transform(spam_email)
model.predict(spam_email_count)

array([1, 1, 1], dtype=int64)

In [12]:
#test model 
X_test_count = cv.transform(X_test)
model.score(X_test_count, y_test)*100.0


97.84637473079684

In [13]:
#KNN

# from sklearn.neighbors import KNeighborsClassifier
# k= 3
# model = KNeighborsClassifier(n_neighbors=k)
# model.fit(X_train_count, y_train)

In [14]:
# emails = ['SĂN DEAL HOT LG MỪNG BLACK FRIDAY']
# emails_count = cv.transform(emails)
# model.predict(emails_count)


In [15]:
# x_test_count = cv.transform(X_test)
# model.score(x_test_count, y_test)
